In [50]:
# Import pandas library for data manipulation and analysis
import pandas as pd 
# Import numpy library for numerical operations
import numpy as np

In [52]:
# Read the CSV file named "orders (1).csv" into a pandas DataFrame
df=pd.read_csv("orders (1).csv")
# Display a random sample of 4 rows from the DataFrame to get a quick overview of the data
df.sample(4)

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
11,16/12/21 10:46,SELL,ASHOKLEY,MIS,2000/2000,125.95,COMPLETE
7,16/12/21 12:39,SELL,ASHOKLEY,MIS,1000/1000,125.96,COMPLETE
8,16/12/21 12:29,BUY,ASHOKLEY,MIS,2000/2000,125.70,COMPLETE
9,16/12/21 11:22,SELL,ASHOKLEY,MIS,2000/2000,125.95,COMPLETE


### DATA CLEANING

In [53]:
# Filter the dataframe to only include rows where:
# 1. "Product" column equals "MIS" AND
# 2. "Status" column equals "COMPLETE"
# This creates a subset of the original dataframe with only completed MIS products
fl_data=df[(df["Product"]=="MIS")&(df["Status"]=="COMPLETE")]
# Display the first 2 rows of the filtered dataframe to inspect the data
fl_data.head(2)

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
1,16/12/21 15:08,BUY,ASHOKLEY,MIS,1000/1000,125.70,COMPLETE
3,16/12/21 14:13,BUY,TATAMOTORS,MIS,250/250,490.55,COMPLETE


In [55]:
# Extract the quantity used from the "Qty." column by splitting on "/" and taking the first part
# Convert the extracted value to an integer and store it in a new column "qty_used"
fl_data.loc[:,"qty_used"]=fl_data["Qty."].apply(lambda x:int(x.split("/")[0]))

### PROBLEMS

### 1) Different types of charges for Individual trade

In [56]:
# Display the fl_data DataFrame to examine its contents
fl_data

,Time,Type,Instrument,Product,Qty.,Avg. price,Status,qty_used
1,16/12/21 15:08,BUY,ASHOKLEY,MIS,1000/1000,125.70,COMPLETE,1000
3,16/12/21 14:13,BUY,TATAMOTORS,MIS,250/250,490.55,COMPLETE,250
5,16/12/21 13:21,SELL,TATAMOTORS,MIS,250/250,492.10,COMPLETE,250
7,16/12/21 12:39,SELL,ASHOKLEY,MIS,1000/1000,125.96,COMPLETE,1000
8,16/12/21 12:29,BUY,ASHOKLEY,MIS,2000/2000,125.70,COMPLETE,2000
9,16/12/21 11:22,SELL,ASHOKLEY,MIS,2000/2000,125.95,COMPLETE,2000
11,16/12/21 10:46,SELL,ASHOKLEY,MIS,2000/2000,125.95,COMPLETE,2000
12,16/12/21 10:24,BUY,ASHOKLEY,MIS,1000/1000,125.60,COMPLETE,1000
13,16/12/21 10:23,BUY,ASHOKLEY,MIS,1000/1000,125.65,COMPLETE,1000


In [58]:
# Calculate the Turnover by multiplying the average price by the quantity used
# This creates a new column 'Turnover' in the fl_data DataFrame
fl_data.loc[:,"Turnover"]=fl_data["Avg. price"]*fl_data["qty_used"]

In [60]:
# Calculate brokerage fee as 0.03% of turnover, with a maximum cap of 20
# The result is stored in a new column called "Brokerage"
fl_data.loc[:,"Brokerage"]=(fl_data["Turnover"]*0.03/100).clip(upper=20)

In [62]:
# Calculate Securities Transaction Tax (STT) or Commodity Transaction Tax (CTT)
# Apply 0.025% tax on turnover for SELL transactions, 0 for other transaction types
fl_data.loc[:,"STT/CTT"]=np.where(fl_data["Type"]=="SELL",fl_data["Turnover"]*0.025/100,0)

In [64]:
# Calculate ETC (Estimated Transaction Cost) as 0.00297% of Turnover
# This creates a new column 'ETC' in the fl_data DataFrame
fl_data.loc[:,"ETC"]= fl_data["Turnover"]*0.00297/100

In [66]:
# Calculate SEBI charges as 0.0001% of turnover (0.0001/100 = 0.000001 or 1 millionth of turnover)
fl_data.loc[:,"SEBI"]= fl_data["Turnover"]*0.0001/100

In [69]:
# Calculate GST (18%) on the sum of Brokerage, ETC, and SEBI fees
fl_data.loc[:,"GST"]=(fl_data["Brokerage"]+fl_data["ETC"]+fl_data["SEBI"])*18/100

In [71]:
# Calculate stamp charges (0.003%) for BUY transactions only
# For BUY transactions: 0.003% of turnover value
# For other transactions: 0 (no stamp charges)
fl_data.loc[:,"Stamp charges"]=np.where(fl_data["Type"]=="BUY",fl_data["Turnover"]*0.003/100,0)

In [73]:
# Calculate the total charges by summing up all individual fee components
# This creates a new column 'Total Charges' that represents the sum of all trading fees
fl_data.loc[:,"Total Charges"] = (
    fl_data["Brokerage"] +     # Brokerage fees charged by the broker
    fl_data["STT/CTT"] +       # Securities/Commodities Transaction Tax
    fl_data["ETC"] +           # Exchange Transaction Charges
    fl_data["SEBI"] +          # Securities and Exchange Board of India fees
    fl_data["GST"] +           # Goods and Services Tax
    fl_data["Stamp charges"]   # Stamp duty charges for transactions
)

In [74]:
# Extract and round financial transaction data to 2 decimal places
# This creates a new DataFrame 'charges' with selected columns from 'fl_data'
# Columns include transaction details (Type, Instrument, Product, etc.) and various fees/charges
charges=fl_data[["Type","Instrument","Product","qty_used","Avg. price","Status","Turnover","Brokerage",
                 "STT/CTT","ETC","SEBI","GST","Stamp charges",
                 "Total Charges"]].round(2)
charges  # Display the charges DataFrame

,Type,Instrument,Product,qty_used,Avg. price,Status,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp charges,Total Charges
1,BUY,ASHOKLEY,MIS,1000,125.70,COMPLETE,125700.0,20.0,0.00,3.73,0.13,4.29,3.77,31.92
3,BUY,TATAMOTORS,MIS,250,490.55,COMPLETE,122637.5,20.0,0.00,3.64,0.12,4.28,3.68,31.72
5,SELL,TATAMOTORS,MIS,250,492.10,COMPLETE,123025.0,20.0,30.76,3.65,0.12,4.28,0.00,58.81
7,SELL,ASHOKLEY,MIS,1000,125.96,COMPLETE,125960.0,20.0,31.49,3.74,0.13,4.30,0.00,59.65
8,BUY,ASHOKLEY,MIS,2000,125.70,COMPLETE,251400.0,20.0,0.00,7.47,0.25,4.99,7.54,40.25
9,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20.0,62.98,7.48,0.25,4.99,0.00,95.70
11,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20.0,62.98,7.48,0.25,4.99,0.00,95.70
12,BUY,ASHOKLEY,MIS,1000,125.60,COMPLETE,125600.0,20.0,0.00,3.73,0.13,4.29,3.77,31.92
13,BUY,ASHOKLEY,MIS,1000,125.65,COMPLETE,125650.0,20.0,0.00,3.73,0.13,4.29,3.77,31.92


### 2) Stock wise and Type wise analysis with weighted Avg. price & calculated charges

In [75]:
# Group the financial data by 'Instrument' and 'Type' columns
# Calculate the sum of various financial metrics for each group
# as_index=False keeps 'Instrument' and 'Type' as columns rather than making them the index
Stock_Summary=fl_data.groupby(["Instrument","Type"],as_index=False)[["qty_used","Avg. price",
                                                                    "Turnover","Brokerage","STT/CTT","ETC","SEBI","GST",
                                                                    "Stamp charges","Total Charges"]].sum()
Stock_Summary  # Display the summary dataframe with aggregated financial data by instrument and type

,Instrument,Type,qty_used,Avg. price,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp charges,Total Charges
0,ASHOKLEY,BUY,5000,502.65,628350.0,80.0,0.00000,18.661995,0.628350,17.872262,18.850500,136.013107
1,ASHOKLEY,SELL,5000,377.86,629760.0,60.0,157.44000,18.703872,0.629760,14.280054,0.000000,251.053686
2,TATAMOTORS,BUY,250,490.55,122637.5,20.0,0.00000,3.642334,0.122637,4.277695,3.679125,31.721791
3,TATAMOTORS,SELL,250,492.10,123025.0,20.0,30.75625,3.653843,0.123025,4.279836,0.000000,58.812954


In [76]:
# Calculate the weighted average price by dividing the total turnover by the quantity used
Stock_Summary["Weighted_Avg_Price"] = Stock_Summary["Turnover"] / Stock_Summary["qty_used"]
# Display the Stock_Summary DataFrame with the newly added column
Stock_Summary

,Instrument,Type,qty_used,Avg. price,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp charges,Total Charges,Weighted_Avg_Price
0,ASHOKLEY,BUY,5000,502.65,628350.0,80.0,0.00000,18.661995,0.628350,17.872262,18.850500,136.013107,125.670
1,ASHOKLEY,SELL,5000,377.86,629760.0,60.0,157.44000,18.703872,0.629760,14.280054,0.000000,251.053686,125.952
2,TATAMOTORS,BUY,250,490.55,122637.5,20.0,0.00000,3.642334,0.122637,4.277695,3.679125,31.721791,490.550
3,TATAMOTORS,SELL,250,492.10,123025.0,20.0,30.75625,3.653843,0.123025,4.279836,0.000000,58.812954,492.100


In [77]:
# Round all numeric values in the Stock_Summary dataframe to 2 decimal places
# and assign the result to a new variable Stock_Type_Summary
Stock_Type_Summary=Stock_Summary.round(2)
# Display the Stock_Type_Summary dataframe
Stock_Type_Summary

,Instrument,Type,qty_used,Avg. price,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp charges,Total Charges,Weighted_Avg_Price
0,ASHOKLEY,BUY,5000,502.65,628350.0,80.0,0.00,18.66,0.63,17.87,18.85,136.01,125.67
1,ASHOKLEY,SELL,5000,377.86,629760.0,60.0,157.44,18.70,0.63,14.28,0.00,251.05,125.95
2,TATAMOTORS,BUY,250,490.55,122637.5,20.0,0.00,3.64,0.12,4.28,3.68,31.72,490.55
3,TATAMOTORS,SELL,250,492.10,123025.0,20.0,30.76,3.65,0.12,4.28,0.00,58.81,492.10


### 3) Overall Summary of each Stocks

In [78]:
# Filter the Stock_Summary dataframe to only include rows where "Type" is "SELL"
# This creates a new dataframe containing only sell transactions
sell_type=Stock_Summary[Stock_Summary["Type"]=="SELL"]
# Display the filtered dataframe containing only sell transactions
sell_type

,Instrument,Type,qty_used,Avg. price,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp charges,Total Charges,Weighted_Avg_Price
1,ASHOKLEY,SELL,5000,377.86,629760.0,60.0,157.44000,18.703872,0.629760,14.280054,0.0,251.053686,125.952
3,TATAMOTORS,SELL,250,492.10,123025.0,20.0,30.75625,3.653843,0.123025,4.279836,0.0,58.812954,492.100


In [79]:
# Filter the Stock_Summary dataframe to only include rows where "Type" is "BUY"
# This creates a new dataframe containing only purchase transactions
buy_type = Stock_Summary[Stock_Summary["Type"] == "BUY"]
# Display the filtered dataframe containing only buy transactions
buy_type

,Instrument,Type,qty_used,Avg. price,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp charges,Total Charges,Weighted_Avg_Price
0,ASHOKLEY,BUY,5000,502.65,628350.0,80.0,0.0,18.661995,0.628350,17.872262,18.850500,136.013107,125.67
2,TATAMOTORS,BUY,250,490.55,122637.5,20.0,0.0,3.642334,0.122637,4.277695,3.679125,31.721791,490.55


In [80]:
# Merge sell and buy transaction data for each stock instrument
# This creates a summary dataframe that combines:
# - Sell data (quantity, average price, total charges)
# - Buy data (quantity, average price, total charges)
# The merge is performed on the "Instrument" column (stock symbol)
# Suffixes "_sell" and "_buy" are added to distinguish between sell and buy columns
Overall_Summary_each_Stocks=pd.merge(sell_type[["Instrument","qty_used","Weighted_Avg_Price","Total Charges"]],
         buy_type[["Instrument","qty_used","Weighted_Avg_Price","Total Charges"]],
         on="Instrument",
         suffixes=("_sell","_buy")
)
Overall_Summary_each_Stocks

,Instrument,qty_used_sell,Weighted_Avg_Price_sell,Total Charges_sell,qty_used_buy,Weighted_Avg_Price_buy,Total Charges_buy
0,ASHOKLEY,5000,125.952,251.053686,5000,125.67,136.013107
1,TATAMOTORS,250,492.100,58.812954,250,490.55,31.721791


In [81]:
# Calculate the matched quantity by taking the minimum of sell and buy quantities
# This identifies how much of each stock was successfully matched between buy and sell orders
Overall_Summary_each_Stocks["match_qty"]=Overall_Summary_each_Stocks[["qty_used_sell","qty_used_buy"]].min(axis=1)

In [82]:
# Access the Overall_Summary_each_Stocks DataFrame which likely contains aggregated metrics
# for each stock in the analysis, such as returns, volatility, or other statistical measures
Overall_Summary_each_Stocks

,Instrument,qty_used_sell,Weighted_Avg_Price_sell,Total Charges_sell,qty_used_buy,Weighted_Avg_Price_buy,Total Charges_buy,match_qty
0,ASHOKLEY,5000,125.952,251.053686,5000,125.67,136.013107,5000
1,TATAMOTORS,250,492.100,58.812954,250,490.55,31.721791,250


In [84]:
# Calculate Gross Profit and Loss (PnL) for each stock
# Formula: (Selling Price - Buying Price) * Quantity
# Uses weighted average prices to account for multiple transactions
Overall_Summary_each_Stocks["Gross PnL"]=(Overall_Summary_each_Stocks["Weighted_Avg_Price_sell"]-
                                          Overall_Summary_each_Stocks["Weighted_Avg_Price_buy"])*Overall_Summary_each_Stocks["match_qty"]

In [86]:
# Calculate the total charges by summing up the selling and buying charges for each stock
Overall_Summary_each_Stocks["Total charges"] = Overall_Summary_each_Stocks["Total Charges_sell"] +Overall_Summary_each_Stocks["Total Charges_buy"]

In [87]:
# Calculate Net Profit and Loss by subtracting Total charges from Gross PnL for each stock
Overall_Summary_each_Stocks["Net PnL"] = Overall_Summary_each_Stocks["Gross PnL"] - Overall_Summary_each_Stocks["Total charges"]

In [90]:
Overall_Summary_each_Stocks["% Charges on Gross PnL"]=(Overall_Summary_each_Stocks["Total charges"]/Overall_Summary_each_Stocks["Gross PnL"])*100

In [91]:
Overall_Summary=Overall_Summary_each_Stocks[["Instrument","Gross PnL","Total charges","Net PnL","% Charges on Gross PnL"]].round(2)

In [93]:
Overall_Summary

,Instrument,Gross PnL,Total charges,Net PnL,% Charges on Gross PnL
0,ASHOKLEY,1410.0,387.07,1022.93,27.45
1,TATAMOTORS,387.5,90.53,296.97,23.36


In [95]:
charges

,Type,Instrument,Product,qty_used,Avg. price,Status,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp charges,Total Charges
1,BUY,ASHOKLEY,MIS,1000,125.70,COMPLETE,125700.0,20.0,0.00,3.73,0.13,4.29,3.77,31.92
3,BUY,TATAMOTORS,MIS,250,490.55,COMPLETE,122637.5,20.0,0.00,3.64,0.12,4.28,3.68,31.72
5,SELL,TATAMOTORS,MIS,250,492.10,COMPLETE,123025.0,20.0,30.76,3.65,0.12,4.28,0.00,58.81
7,SELL,ASHOKLEY,MIS,1000,125.96,COMPLETE,125960.0,20.0,31.49,3.74,0.13,4.30,0.00,59.65
8,BUY,ASHOKLEY,MIS,2000,125.70,COMPLETE,251400.0,20.0,0.00,7.47,0.25,4.99,7.54,40.25
9,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20.0,62.98,7.48,0.25,4.99,0.00,95.70
11,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20.0,62.98,7.48,0.25,4.99,0.00,95.70
12,BUY,ASHOKLEY,MIS,1000,125.60,COMPLETE,125600.0,20.0,0.00,3.73,0.13,4.29,3.77,31.92
13,BUY,ASHOKLEY,MIS,1000,125.65,COMPLETE,125650.0,20.0,0.00,3.73,0.13,4.29,3.77,31.92


In [94]:
Stock_Type_Summary

,Instrument,Type,qty_used,Avg. price,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp charges,Total Charges,Weighted_Avg_Price
0,ASHOKLEY,BUY,5000,502.65,628350.0,80.0,0.00,18.66,0.63,17.87,18.85,136.01,125.67
1,ASHOKLEY,SELL,5000,377.86,629760.0,60.0,157.44,18.70,0.63,14.28,0.00,251.05,125.95
2,TATAMOTORS,BUY,250,490.55,122637.5,20.0,0.00,3.64,0.12,4.28,3.68,31.72,490.55
3,TATAMOTORS,SELL,250,492.10,123025.0,20.0,30.76,3.65,0.12,4.28,0.00,58.81,492.10


### TRANSFORMING INSIGHTS INTO EXCEL

In [96]:
dfs = [charges,Stock_Type_Summary,Overall_Summary]
sheet_names = ["charges","Stock_Type_Summary","Overall_Summary"]

with pd.ExcelWriter("Kite PnL.xlsx") as writer:
    for df, name in zip(dfs, sheet_names):
        df.to_excel(writer, sheet_name=name, index=False)